In [ ]:
# !pip install langchain langchain-openai langgraph chromadb 

In [1]:
from typing import List, TypedDict, Annotated, Optional, Dict
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Memory stores
from memory_stores.semantic_memory import SemanticMemoryStore
from memory_stores.episodic_memory import EpisodicMemoryStore
from memory_stores.preference_store import PreferenceMemoryStore
from memory_stores.procedural_memory import ProceduralMemory

# Node modules import
import nodes.semantic_read as semantic_read_module
import nodes.episodic_read as episodic_read_module
import nodes.preference_read as preference_read_module
import nodes.procedural_guard as procedural_guard_module
import nodes.planner as planner_module
import nodes.tool_node as tool_node_module
import nodes.response as response_module
import nodes.salience_gated_memory_write as memory_write_module
import nodes.conflict_resolution as conflict_resolution_module

# Utils
from utils.salience_scoring import SalienceScorer

# Reload modules if code was updated (run this cell after updating node files)
import importlib
importlib.reload(planner_module)

C:\Users\varsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'nodes.planner' from 'c:\\Users\\varsh\\OneDrive\\Documents\\study\\blogs\\Agentic-AI\\Memory\\lab_3_advanced_memory_techniques\\nodes\\planner.py'>

In [ ]:
# Initialize LLM and embeddings
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Initialize memory stores
semantic_store = SemanticMemoryStore(embeddings)
episodic_store = EpisodicMemoryStore(embeddings)
procedural_memory = ProceduralMemory()
preference_store = PreferenceMemoryStore()

# Initialize salience scorer
salience_scorer = SalienceScorer(llm)

# Set namespace
namespace = "crm_support/coddy001"

# Configure nodes with shared state
def configure_nodes():
    """Configure all node modules with shared dependencies."""
    # Memory read nodes
    semantic_read_module.namespace = namespace
    semantic_read_module.semantic_store = semantic_store
    
    episodic_read_module.namespace = namespace
    episodic_read_module.episodic_store = episodic_store
    
    preference_read_module.namespace = namespace
    preference_read_module.preference_store = preference_store
    
    # LLM-based nodes
    planner_module.llm = llm
    procedural_guard_module.llm = llm
    response_module.llm = llm
    
    # Memory write node (needs multiple dependencies)
    memory_write_module.namespace = namespace
    memory_write_module.llm = llm
    memory_write_module.salience_scorer = salience_scorer
    memory_write_module.semantic_store = semantic_store
    memory_write_module.episodic_store = episodic_store
    
    # Conflict resolution node
    conflict_resolution_module.namespace = namespace
    conflict_resolution_module.semantic_store = semantic_store

configure_nodes()

#### Graph Definition


In [3]:
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    tool_result: Optional[dict]
    semantic_memories: Optional[List[Dict]]
    episodic_memories: Optional[List[Dict]]
    preferences: Optional[Dict]
    escalation_info: Optional[Dict]
    selected_procedure: Optional[str]
    procedure_reason: Optional[str]
    planner_action: Optional[str]
    planner_tool: Optional[str]
    planner_arguments: Optional[Dict]

In [4]:
checkpointer = MemorySaver()
graph = StateGraph(AgentState)

# Add nodes in order
graph.add_node("semantic_read", semantic_read_module.semantic_read)
graph.add_node("episodic_read", episodic_read_module.episodic_read)
graph.add_node("preference_read", preference_read_module.preference_read)
graph.add_node("planner", planner_module.planner_node)
graph.add_node("procedural_guard", procedural_guard_module.procedural_guard)
graph.add_node("tool", tool_node_module.tool_node)
graph.add_node("conflict_resolution", conflict_resolution_module.conflict_resolution)
graph.add_node("respond", response_module.response_node)
graph.add_node("salience_gated_memory_write", memory_write_module.salience_gated_memory_write)

# Set entry point
graph.set_entry_point("semantic_read")

# Wire edges
graph.add_edge("semantic_read", "episodic_read")
graph.add_edge("episodic_read", "preference_read")
graph.add_edge("preference_read", "planner")
graph.add_edge("planner", "procedural_guard")
graph.add_edge("procedural_guard", "tool")
# After tool execution, go to conflict resolution, then respond
graph.add_edge("tool", "conflict_resolution")
graph.add_edge("conflict_resolution", "respond")
# After respond, write memories
graph.add_edge("respond", "salience_gated_memory_write")
graph.add_edge("salience_gated_memory_write", END)

app = graph.compile(checkpointer=checkpointer)
png_data = app.get_graph().draw_mermaid_png()

with open("lab3_flow.png", "wb") as f:
    f.write(png_data)


#### Chat definition

In [5]:
def run_chat(inputs, thread_id="default_thread"):
    """Run chat with persistent state across calls."""
    for text in inputs:
        print(f"\n🧑 USER: {text}")
        out = app.invoke(
            {"messages": [HumanMessage(content=text)]},
            {"configurable": {"thread_id": thread_id}}
        )
        print(f"🤖 AGENT: {out['messages'][-1].content}")
        print("=" * 60)
    return out


### Demos

#### Demo 1: Session 1 (Day 1) - Initial Ticket Creation & Memory Storage
**Capabilities shown:**
- Semantic memory storage (facts: ticket ID, speed plan, modem/router models, customer name)
- Episodic memory storage (experiences: troubleshooting attempts)
- Procedure selection (LLM choosing appropriate workflow)
- Salience-gated writes (only important information stored)


In [6]:
session1 = run_chat([
    "Hi, I'm Cody. I'm experiencing slow internet speeds - my connection is much slower than my plan promises. Can you create a support ticket?",
    "I've tried restarting my modem and router multiple times, but the speeds are still very slow.",
    "I have a 500 Mbps plan but I'm only getting around 50 Mbps. My modem is a Netgear CM1000 and my router is a TP-Link Archer AX50. This started happening about 3 days ago.",
    "Can you tell me my ticket number?"
], thread_id="session1")




🧑 USER: Hi, I'm Cody. I'm experiencing slow internet speeds - my connection is much slower than my plan promises. Can you create a support ticket?
🤖 AGENT: I've created a support ticket for you, Cody. Your ticket number is **998884**. 

To help resolve the slow internet speeds, could you please provide the model of your device? Additionally, if you have noticed any specific symptoms or patterns with the slow speeds, that information would be helpful.

🧑 USER: I've tried restarting my modem and router multiple times, but the speeds are still very slow.
🤖 AGENT: I've updated your ticket (998884) with the information that you've restarted your modem and router multiple times, but the speeds are still slow.

As the next step, could you please provide the model of your device? This will help in troubleshooting the issue further.

🧑 USER: I have a 500 Mbps plan but I'm only getting around 50 Mbps. My modem is a Netgear CM1000 and my router is a TP-Link Archer AX50. This started happening ab

### Demo 2: Session 2 (Day 2) - Cross-Session Memory Recall
**Capabilities shown:**
- Semantic memory recall (facts persist across sessions)
- Episodic memory recall (past experiences remembered)
- Multi-memory integration (semantic + episodic working together)
- Fresh thread_id but memories persist


In [7]:
session2 = run_chat([
    "What's the status of my ticket?",
    "What speed plan do I have again?",
    "Is there any other troubleshooting we could try?"
], thread_id="session2")

print("\n" + "=" * 80)
print("Session 2 complete. Agent recalled information from Session 1!")
print("=" * 80)



🧑 USER: What's the status of my ticket?
🤖 AGENT: Your ticket (ID: 998884) is currently in "New" status. You reported slow internet speeds, stating that you have a 500 Mbps plan but are only getting around 50 Mbps. You also mentioned that you have a Netgear CM1000 modem and a TP-Link Archer AX50 router, and that this issue started about three days ago.

Since you've already tried restarting your modem and router multiple times, I recommend checking the following:

1. **Wiring**: Ensure all cables are securely connected and not damaged.
2. **Network Congestion**: Check if multiple devices are using the internet simultaneously, which could slow down speeds.
3. **Firmware Updates**: Make sure your modem and router firmware are up to date.

If the issue persists after these steps, please let me know, and we can escalate the ticket for further assistance.

🧑 USER: What speed plan do I have again?
🤖 AGENT: You have a 500 Mbps speed plan. If you have any further questions or need assistance, 

### Demo 3: Session 3 (Day 7) - Long-term Memory Persistence
**Capabilities shown:**
- Long-term memory persistence (memories survive over time)
- Episodic memory with recency bias (recent memories prioritized)
- Semantic memory consolidation (facts remain accurate)


In [ ]:
session3 = run_chat([
    "I had an internet speed problem last week. Can you remind me what it was?",
    "What's the latest on my ticket?",
    "What speed plan do I have again?"
], thread_id="session3")

print("\n" + "=" * 80)
print("Session 3 complete. Long-term memories successfully recalled!")
print("=" * 80)


## Inspect Stored Memories

### View Semantic Memories


In [ ]:
# View semantic memories (facts, domain knowledge)
semantic_results = semantic_store.search(namespace, "ticket email device customer", top_k=10)
print(f"\nFound {len(semantic_results)} semantic memories:\n")
for i, r in enumerate(semantic_results, 1):
    print(f"{i}. {r['content']}")
    print(f"   Key: {r['metadata'].get('key', 'N/A')}")
    print(f"   Timestamp: {r['metadata'].get('timestamp', 'N/A')}")
    print()


### View Episodic Memories


In [ ]:
# View episodic memories (experiences, past interactions)
print("=" * 80)
print("EPISODIC MEMORIES (Experiences, Past Interactions)")
print("=" * 80)
episodic_results = episodic_store.search(namespace, "troubleshooting issue problem", top_k=10)
print(f"\nFound {len(episodic_results)} episodic memories:\n")
for i, r in enumerate(episodic_results, 1):
    print(f"{i}. {r['content']}")
    if 'combined_score' in r:
        print(f"   Score: {r['combined_score']:.3f} (similarity: {r.get('similarity', 0):.3f}, recency: {r.get('recency_score', 0):.3f})")
    print()


In [ ]:
# View episodic memories
episodic_results = episodic_store.search(namespace, "troubleshooting tried", top_k=5)
print(f"Found {len(episodic_results)} episodic memories:\n")
for r in episodic_results:
    print(f"- {r['content']}")
    print(f"  Score: {r.get('combined_score', 0):.3f} (similarity: {r.get('similarity', 0):.3f}, recency: {r.get('recency_score', 0):.3f})\n")


#### Detailed memory behind the scenes - just for reference

In [10]:
def run_chat_detailed(inputs, thread_id="default_thread"):
    """Run chat with detailed memory and execution state inspection."""
    import sys
    import json
    
    for text in inputs:
        print(f"\nUSER: {text}")
        print("\n" + "="*80)
        print("GRAPH EXECUTION - NODE BY NODE")
        print("="*80)
        
        final_state = None
        
        for event in app.stream(
            {"messages": [HumanMessage(content=text)]},
            {"configurable": {"thread_id": thread_id}}
        ):
            for node_name, state_update in event.items():
                print(f"\nNODE: {node_name}")
                
                # Semantic Memory Read
                if node_name == "semantic_read":
                    memories = state_update.get("semantic_memories", [])
                    print(f"  Semantic Memories Found: {len(memories)}")
                    if memories:
                        print("  Content:")
                        for i, mem in enumerate(memories[:3], 1):
                            content = mem.get('content', str(mem))
                            print(f"    {i}. {content[:150]}{'...' if len(content) > 150 else ''}")
                
                # Episodic Memory Read
                if node_name == "episodic_read":
                    memories = state_update.get("episodic_memories", [])
                    print(f"  Episodic Memories Found: {len(memories)}")
                    if memories:
                        print("  Content:")
                        for i, mem in enumerate(memories[:3], 1):
                            content = mem.get('content', str(mem))
                            print(f"    {i}. {content[:150]}{'...' if len(content) > 150 else ''}")
                
                # Preference Read
                if node_name == "preference_read":
                    prefs = state_update.get("preferences", {})
                    print(f"  Preferences Found: {len(prefs)}")
                    if prefs:
                        print(f"  Keys: {list(prefs.keys())}")
                
                # Planner
                if node_name == "planner":
                    print("  Planner Decision:")
                    for msg in state_update.get("messages", []):
                        if hasattr(msg, 'content') and isinstance(msg.content, str):
                            content = msg.content.strip()
                            if content.startswith('{'):
                                try:
                                    plan = json.loads(content)
                                    print(f"    Procedure: {plan.get('procedure', 'N/A')}")
                                    print(f"    Tool: {plan.get('tool', 'N/A')}")
                                    args = plan.get('arguments', {})
                                    if args:
                                        print(f"    Arguments:")
                                        for key, value in args.items():
                                            print(f"      {key}: {value}")
                                    else:
                                        print(f"    Arguments: (empty)")
                                except:
                                    print(f"    Raw JSON: {content[:200]}...")
                    
                    procedure = state_update.get('selected_procedure', 'N/A')
                    tool = state_update.get('planner_tool', 'N/A')
                    print(f"  Selected Procedure: {procedure}")
                    print(f"  Selected Tool: {tool}")
                    args = state_update.get('planner_arguments', {})
                    if args:
                        print(f"  Extracted Arguments:")
                        for key, value in args.items():
                            print(f"    {key}: {value}")
                
                # Tool Execution
                if node_name == "tool":
                    tool_result = state_update.get("tool_result")
                    if tool_result:
                        if "ticket_id" in tool_result:
                            print(f"  Tool Result: Success")
                            print(f"    ticket_id: {tool_result['ticket_id']}")
                            if "status" in tool_result:
                                print(f"    status: {tool_result['status']}")
                        elif "error" in tool_result:
                            print(f"  Tool Result: Error")
                            print(f"    Error: {tool_result['error']}")
                        else:
                            print(f"  Tool Result: {list(tool_result.keys())}")
                    else:
                        print(f"  Tool Result: None")
                
                # Memory Write (Salience-Gated)
                if node_name == "salience_gated_memory_write":
                    print("  Salience-Gated Memory Write:")
                    
                    # Use accumulated state from previous nodes
                    current_state = final_state if final_state else state_update
                    if current_state:
                        recent = current_state.get("messages", [])[-6:] if len(current_state.get("messages", [])) >= 6 else current_state.get("messages", [])
                        conversation = "\n".join([f"{type(m).__name__}: {m.content}" for m in recent])
                        tool_result = current_state.get("tool_result")
                        
                        # Check for explicit triggers
                        explicit_trigger = False
                        if tool_result:
                            if "ticket_id" in tool_result and tool_result.get("status") in ["created", "updated"]:
                                explicit_trigger = True
                            if "ticket" in tool_result and isinstance(tool_result.get("ticket"), dict) and tool_result["ticket"].get("status") in ["Escalated", "Resolved"]:
                                explicit_trigger = True
                        
                        # Compute salience scores
                        scores = salience_scorer.compute_salience(conversation, tool_result)
                        combined_score = (
                            0.4 * scores["importance"] +
                            0.3 * scores["novelty"] +
                            0.2 * scores["contradiction"] -
                            0.1 * scores["risk"]
                        )
                        should_write = salience_scorer.should_write(scores, threshold=0.6, explicit_trigger=explicit_trigger)
                        
                        print("    Salience Scores:")
                        print(f"      importance: {scores['importance']:.2f}")
                        print(f"      novelty: {scores['novelty']:.2f}")
                        print(f"      contradiction: {scores['contradiction']:.2f}")
                        print(f"      risk: {scores['risk']:.2f}")
                        print(f"    Combined Score: {combined_score:.2f} (threshold: 0.6)")
                        print(f"    Explicit Trigger: {explicit_trigger}")
                        print(f"    Should Write: {should_write}")
                        print("    Process:")
                        print("      1. Extracts semantic facts (ticket IDs, devices, customer info)")
                        print("      2. Extracts episodic experiences (troubleshooting attempts, interactions)")
                        print("      3. Writes to semantic_store and episodic_store")
                    else:
                        print("    (Unable to compute scores - state not available)")
                
                # Update final_state with current state_update (merge if needed)
                if state_update:
                    if final_state:
                        # Merge state updates
                        for key, value in state_update.items():
                            if value:  # Only update if value is not None/empty
                                final_state[key] = value
                    else:
                        final_state = state_update
                elif not final_state:
                    final_state = state_update
                sys.stdout.flush()
        
        print("\n" + "="*80)
        print("FINAL RESPONSE")
        print("="*80)
        
        if final_state and "messages" in final_state:
            for msg in reversed(final_state["messages"]):
                if hasattr(msg, 'content'):
                    content = msg.content
                    if not (content.strip().startswith('{') and ('"procedure"' in content or '"tool"' in content)):
                        print(f"AGENT: {content}")
                        break
        else:
            out = app.invoke(
                {"messages": [HumanMessage(content=text)]},
                {"configurable": {"thread_id": thread_id}}
            )
            print(f"AGENT: {out['messages'][-1].content}")
        
        print("=" * 60)
        sys.stdout.flush()
    
    return final_state if final_state else {}


In [11]:
session1 = run_chat_detailed([
    "Hi, I'm Cody. I'm experiencing slow internet speeds - my connection is much slower than my plan promises. Can you create a support ticket?",
    "I've tried restarting my modem and router multiple times, but the speeds are still very slow.",
    "I have a 500 Mbps plan but I'm only getting around 50 Mbps. My modem is a Netgear CM1000 and my router is a TP-Link Archer AX50. This started happening about 3 days ago.",
    "Can you tell me my ticket number?"
], thread_id="session1")




USER: Hi, I'm Cody. I'm experiencing slow internet speeds - my connection is much slower than my plan promises. Can you create a support ticket?

GRAPH EXECUTION - NODE BY NODE

NODE: semantic_read
  Semantic Memories Found: 0

NODE: episodic_read
  Episodic Memories Found: 0

NODE: preference_read
  Preferences Found: 0

NODE: planner
  Planner Decision:
  Selected Procedure: standard_support
  Selected Tool: N/A

NODE: procedural_guard

NODE: tool
  Tool Result: Success
    ticket_id: 998885
    status: created

NODE: conflict_resolution

NODE: respond

NODE: salience_gated_memory_write
  Salience-Gated Memory Write:
    Salience Scores:
      importance: 1.00
      novelty: 0.00
      contradiction: 0.00
      risk: 0.00
    Combined Score: 0.40 (threshold: 0.6)
    Explicit Trigger: True
    Should Write: True
    Process:
      1. Extracts semantic facts (ticket IDs, devices, customer info)
      2. Extracts episodic experiences (troubleshooting attempts, interactions)
      3. W

In [12]:
#another_session
session2 = run_chat_detailed([
    "What's the status of my ticket?",
    "What speed plan do I have again?",
    "Is there any other troubleshooting we could try?"
], thread_id="session2")

print("\n" + "=" * 80)
print("Session 2 complete. Agent recalled information from Session 1!")
print("=" * 80)



USER: What's the status of my ticket?

GRAPH EXECUTION - NODE BY NODE

NODE: semantic_read
  Semantic Memories Found: 2
  Content:
    1. Ticket 998885 status: New
    2. Customer has a device in an unspecified location. Ticket 998885.

NODE: episodic_read
  Episodic Memories Found: 3
  Content:
    1. Customer reported slow internet speeds. Agent created a support ticket.
    2. Customer reported slow internet speeds. Agent created a support ticket.
    3. Customer reported slow internet speeds. Agent created a support ticket.

NODE: preference_read
  Preferences Found: 0

NODE: planner
  Planner Decision:
  Selected Procedure: quick_resolution
  Selected Tool: N/A

NODE: procedural_guard

NODE: tool
  Tool Result: Success
    ticket_id: 998885

NODE: conflict_resolution

NODE: respond

NODE: salience_gated_memory_write
  Salience-Gated Memory Write:
    Salience Scores:
      importance: 0.90
      novelty: 0.20
      contradiction: 0.00
      risk: 0.10
    Combined Score: 0.41 (th

#### Conflict Resolution Demo

This demonstrates how the conflict resolution node handles conflicts between tool output and semantic memories. When tool output contradicts existing memories, the tool output is considered authoritative and memories are updated.


In [ ]:
# Conflict Resolution Example
run_chat_detailed([
    "Hi, I'm Cody. what is my ticket status again?"
], thread_id="conflict_demo")

# Then update with different device - this will create a conflict
# The tool output (correct device) will override the memory (old device)
run_chat_detailed([
    "Actually, I made a mistake. My router is a Netgear Nighthawk, not TP-Link."
], thread_id="conflict_demo")



USER: Hi, I'm Cody. what is my ticket status again?

GRAPH EXECUTION - NODE BY NODE

NODE: semantic_read
  Semantic Memories Found: 3
  Content:
    1. Ticket 998885 status: New
    2. Customer has a modem in an unspecified location. Ticket 998885.
    3. Customer has a 500 Mbps internet plan.

NODE: episodic_read
  Episodic Memories Found: 3
  Content:
    1. Customer has a 500 Mbps plan but is only getting around 50 Mbps. This issue started happening about 3 days ago.
    2. Customer reported slow internet speeds. Agent created a support ticket.
    3. Customer reported slow internet speeds. Agent created a support ticket.

NODE: preference_read
  Preferences Found: 0

NODE: planner
  Planner Decision:
  Selected Procedure: quick_resolution
  Selected Tool: N/A

NODE: procedural_guard

NODE: tool
  Tool Result: Success
    ticket_id: 998885

NODE: conflict_resolution

NODE: respond

NODE: salience_gated_memory_write
  Salience-Gated Memory Write:
    Salience Scores:
      importanc